<a href="https://colab.research.google.com/github/mamiri6/EvidenceDetection-NLP/blob/main/spark_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.c

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [3]:
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]
data

[('James', '', 'Smith', '1991-04-01', 'M', 3000),
 ('Michael', 'Rose', '', '2000-05-19', 'M', 4000),
 ('Robert', '', 'Williams', '1978-09-05', 'M', 4000),
 ('Maria', 'Anne', 'Jones', '1967-12-01', 'F', 4000),
 ('Jen', 'Mary', 'Brown', '1980-02-17', 'F', -1)]

In [4]:
columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)

In [5]:
df.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [6]:
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



In [12]:
df.createOrReplaceTempView('MyTable')


In [13]:
d=spark.sql('select * from MyTable')
d.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [14]:
d.select("firstname","gender").show()

+---------+------+
|firstname|gender|
+---------+------+
|    James|     M|
|  Michael|     M|
|   Robert|     M|
|    Maria|     F|
|      Jen|     F|
+---------+------+



In [15]:
d=spark.sql('select * from MyTable')
d.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|    James|          |   Smith|1991-04-01|     M|  3000|
|  Michael|      Rose|        |2000-05-19|     M|  4000|
|   Robert|          |Williams|1978-09-05|     M|  4000|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|
+---------+----------+--------+----------+------+------+



In [16]:
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
d=d.withColumn('dep',lit('ACC'))

In [17]:
d.show()

+---------+----------+--------+----------+------+------+---+
|firstname|middlename|lastname|       dob|gender|salary|dep|
+---------+----------+--------+----------+------+------+---+
|    James|          |   Smith|1991-04-01|     M|  3000|ACC|
|  Michael|      Rose|        |2000-05-19|     M|  4000|ACC|
|   Robert|          |Williams|1978-09-05|     M|  4000|ACC|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|ACC|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|ACC|
+---------+----------+--------+----------+------+------+---+



In [18]:
d=d.withColumn('dep',F.when(F.col('gender')=='M',F.lit('IT')).otherwise(F.lit('ACC')))
d.show()

+---------+----------+--------+----------+------+------+---+
|firstname|middlename|lastname|       dob|gender|salary|dep|
+---------+----------+--------+----------+------+------+---+
|    James|          |   Smith|1991-04-01|     M|  3000| IT|
|  Michael|      Rose|        |2000-05-19|     M|  4000| IT|
|   Robert|          |Williams|1978-09-05|     M|  4000| IT|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|ACC|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|ACC|
+---------+----------+--------+----------+------+------+---+



In [19]:
d=d.withColumn('dep',F.when(F.col('lastname')=='Smith',F.lit('MNG')).otherwise(F.col('dep')))
d.show()

+---------+----------+--------+----------+------+------+---+
|firstname|middlename|lastname|       dob|gender|salary|dep|
+---------+----------+--------+----------+------+------+---+
|    James|          |   Smith|1991-04-01|     M|  3000|MNG|
|  Michael|      Rose|        |2000-05-19|     M|  4000| IT|
|   Robert|          |Williams|1978-09-05|     M|  4000| IT|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|ACC|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|ACC|
+---------+----------+--------+----------+------+------+---+



In [20]:
params=['ACC','IT']

In [21]:
F.col('dep').rlike('|'.join(params))

Column<b'dep RLIKE ACC|IT'>

In [22]:
dn=d.withColumn('rating',F.when(F.col('dep').rlike('|'.join(params)),\
                                F.lit('High Salary')).otherwise(F.lit('Low Salary')))
dn.show()

+---------+----------+--------+----------+------+------+---+-----------+
|firstname|middlename|lastname|       dob|gender|salary|dep|     rating|
+---------+----------+--------+----------+------+------+---+-----------+
|    James|          |   Smith|1991-04-01|     M|  3000|MNG| Low Salary|
|  Michael|      Rose|        |2000-05-19|     M|  4000| IT|High Salary|
|   Robert|          |Williams|1978-09-05|     M|  4000| IT|High Salary|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|ACC|High Salary|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|ACC|High Salary|
+---------+----------+--------+----------+------+------+---+-----------+



In [24]:
dnn=dn.withColumn('rating',\
                F.when(F.col('dep')=='MNG',F.lit('Fake Low Salary'))\
                .when((F.col('salary')==-1) & (F.col('dep')!='MNG') , F.lit('UNDEFINED'))\
                .when(F.col('lastname') =='Jones' , F.lit('Extemely High Salary'))\
                .otherwise(F.col('rating'))
)
dnn.show() 

+---------+----------+--------+----------+------+------+---+--------------------+
|firstname|middlename|lastname|       dob|gender|salary|dep|              rating|
+---------+----------+--------+----------+------+------+---+--------------------+
|    James|          |   Smith|1991-04-01|     M|  3000|MNG|     Fake Low Salary|
|  Michael|      Rose|        |2000-05-19|     M|  4000| IT|         High Salary|
|   Robert|          |Williams|1978-09-05|     M|  4000| IT|         High Salary|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|ACC|Extemely High Salary|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|ACC|           UNDEFINED|
+---------+----------+--------+----------+------+------+---+--------------------+



In [25]:
e=dnn.filter(F.col('rating')!='High Salary').collect()
for i,row in enumerate(e):
  print(row['firstname'])

James
Maria
Jen


In [26]:
nanValue=dnn.na.drop(how='any')
nanValue.show()

+---------+----------+--------+----------+------+------+---+--------------------+
|firstname|middlename|lastname|       dob|gender|salary|dep|              rating|
+---------+----------+--------+----------+------+------+---+--------------------+
|    James|          |   Smith|1991-04-01|     M|  3000|MNG|     Fake Low Salary|
|  Michael|      Rose|        |2000-05-19|     M|  4000| IT|         High Salary|
|   Robert|          |Williams|1978-09-05|     M|  4000| IT|         High Salary|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|ACC|Extemely High Salary|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|ACC|           UNDEFINED|
+---------+----------+--------+----------+------+------+---+--------------------+



In [28]:
dnn.groupBy(['dep','rating']).agg('sum','salary').show()

AssertionError: ignored

In [95]:
pip install koalas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 390 kB 5.0 MB/s 


In [96]:
import databricks.koalas as ks

In [29]:
#kdf = ks.from_pandas(dnn)
dnn.show()

+---------+----------+--------+----------+------+------+---+--------------------+
|firstname|middlename|lastname|       dob|gender|salary|dep|              rating|
+---------+----------+--------+----------+------+------+---+--------------------+
|    James|          |   Smith|1991-04-01|     M|  3000|MNG|     Fake Low Salary|
|  Michael|      Rose|        |2000-05-19|     M|  4000| IT|         High Salary|
|   Robert|          |Williams|1978-09-05|     M|  4000| IT|         High Salary|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|ACC|Extemely High Salary|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|ACC|           UNDEFINED|
+---------+----------+--------+----------+------+------+---+--------------------+



In [30]:
def cast_all_to_float(df):
  return df.select([F.col(colName).cast("int") for colName in df.columns if colName=='salary'])

def sort_columns_asc(input_df):
    return input_df.select(*sorted(input_df.columns))

#dTT=spark.createDataFrame(dnn)
dnd=dnn.toDF(*['firstname','middlename','lastname','dob','gender','salary','dep','rating'])
dnd.show()
dnd.transform(sort_columns_asc).show()


+---------+----------+--------+----------+------+------+---+--------------------+
|firstname|middlename|lastname|       dob|gender|salary|dep|              rating|
+---------+----------+--------+----------+------+------+---+--------------------+
|    James|          |   Smith|1991-04-01|     M|  3000|MNG|     Fake Low Salary|
|  Michael|      Rose|        |2000-05-19|     M|  4000| IT|         High Salary|
|   Robert|          |Williams|1978-09-05|     M|  4000| IT|         High Salary|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|ACC|Extemely High Salary|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|ACC|           UNDEFINED|
+---------+----------+--------+----------+------+------+---+--------------------+



AttributeError: ignored

In [31]:
v=dnn.select('firstname','lastname','salary').rdd.map(lambda x: ('{} {}'.format(x[0],x[1]),x[2]*2) )
v
vv=v.toDF(['name','salary'])
vv.show()
vv.transform()

+---------------+------+
|           name|salary|
+---------------+------+
|    James Smith|  6000|
|       Michael |  8000|
|Robert Williams|  8000|
|    Maria Jones|  8000|
|      Jen Brown|    -2|
+---------------+------+



AttributeError: ignored

In [115]:


for vv in v:
  print(vv)

(3000, 3000)
(4000, 4000)
(4000, 4000)
(4000, 4000)
(-1, -1)


In [64]:

dnd.groupBy("dep").agg(F.avg("salary").alias("avg"),F.sum("salary").alias("sum")).show()


+---+------+----+
|dep|   avg| sum|
+---+------+----+
|ACC|1999.5|3999|
|MNG|3000.0|3000|
| IT|4000.0|8000|
+---+------+----+



In [65]:
dnd.dropna().show()

+---------+----------+--------+----------+------+------+---+--------------------+
|firstname|middlename|lastname|       dob|gender|salary|dep|              rating|
+---------+----------+--------+----------+------+------+---+--------------------+
|    James|          |   Smith|1991-04-01|     M|  3000|MNG|     Fake Low Salary|
|  Michael|      Rose|        |2000-05-19|     M|  4000| IT|         High Salary|
|   Robert|          |Williams|1978-09-05|     M|  4000| IT|         High Salary|
|    Maria|      Anne|   Jones|1967-12-01|     F|  4000|ACC|Extemely High Salary|
|      Jen|      Mary|   Brown|1980-02-17|     F|    -1|ACC|           UNDEFINED|
+---------+----------+--------+----------+------+------+---+--------------------+

